In [189]:
using MatrixNetworks
using Distributions
using Gadfly
using Clustering
using NPZ
using BenchmarkTools, Compat

########### mu = 0.2 ###########

#LFR1
community0 = readdlm("Data/LFR1/community.dat");
adj = readdlm("Data/LFR1/network.dat");

#LFR2
#community0 = readdlm("Data/LFR2/community.dat");
#adj = readdlm("Data/LFR2/network.dat");

#LFR3
#community0 = readdlm("Data/LFR3/community.dat");
#adj = readdlm("Data/LFR3/network.dat");

#LFR4
#community0 = readdlm("Data/LFR4/community.dat");
#adj = readdlm("Data/LFR4/network.dat");

#LFR5
#community0 = readdlm("Data/LFR5/community.dat");
#adj = readdlm("Data/LFR5/network.dat");

#LFR6
#community0 = readdlm("Data/LFR6/community.dat");
#adj = readdlm("Data/LFR6/network.dat");

#LFR7
#community0 = readdlm("Data/LFR7/community.dat");
#adj = readdlm("Data/LFR7/network.dat");

#LFR8
#community0 = readdlm("Data/LFR8/community.dat");
#adj = readdlm("Data/LFR8/network.dat");

########### mu = 0.5 ###########
#LFR9
#community0 = readdlm("Data/LFR9/community.dat");
#adj = readdlm("Data/LFR9/network.dat");

#LFR10
#community0 = readdlm("Data/LFR10/community.dat");
#adj = readdlm("Data/LFR10/network.dat");

#LFR11
#community0 = readdlm("Data/LFR11/community.dat");
#adj = readdlm("Data/LFR11/network.dat");

#LFR12
#community0 = readdlm("Data/LFR12/community.dat");
#adj = readdlm("Data/LFR12/network.dat");

#LFR13
#community0 = readdlm("Data/LFR13/community.dat");
#adj = readdlm("Data/LFR13/network.dat");

#LFR14
#community0 = readdlm("Data/LFR14/community.dat");
#adj = readdlm("Data/LFR14/network.dat");


community_true = community0[:,2];
N = length(community_true);
n_edge2 = size(adj)[1];

A = sparse(adj[:,1],adj[:,2],vec(ones(Int64,n_edge2,1)),N,N);
A,~ = largest_component(A);

adj = 0;
gc();
d = sum(A,2);
s = sum(d);

In [190]:
r = 80;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[11.9533, 11.7928, 11.5132, 11.4487, 10.6794, 10.3786, 9.93135, 9.77978, 9.75072, 9.50646, 9.4902, 9.33323, 9.33851, 9.2759, 9.16474, 9.12417, 9.08061, 9.05941, 8.90794, 8.85709, 8.86722, 8.8703, 8.66912, 8.67092, 8.63288, 8.57281, 8.52799, 8.51393, 8.50678, 8.45386, 8.39473, 8.38859, 8.30226, 8.25748, 8.17658, 8.20185, 8.19251, 8.09472, 8.05966, 8.0644, 7.99324, 7.9594, 7.99715, 7.93133, 7.88202, 7.83201, 7.83938, 7.82343, 7.78277, 7.81893, 7.72757, 7.63589, 7.67852, 7.64196, 7.60096, 7.53674, 7.53843, 7.51304, 7.47358, 7.49944, 7.45235, 7.34348, 7.41098, 7.38896, 7.31962, 7.26662, 7.22306, 7.21605, 7.15234, 7.17693, 7.15613, 7.11282, 7.07061, 7.10751, 7.08306, 7.0472, 7.0132, 6.96553, 6.97051, 6.93662]

In [192]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Sig = v0[:,1:17]';

n_clusters = 200;
n_it_Kmeans = 1000;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;
# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 8
Number of communities: 11
Modularity: 0.25233601108033243
